<a href="https://colab.research.google.com/github/Azure-107/worldfootball_scrape/blob/main/whole_playingtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import pandas as pd 
import time

In [53]:
eng_name = ['arsenal-fc', 'aston-villa', 'brighton-hove-albion', 'burnley-fc', 'chelsea-fc', 'crystal-palace', 'everton-fc', 'fulham-fc', 'leeds-united', 'leicester-city', 'liverpool-fc', 'manchester-city', 'manchester-united', 'newcastle-united', 'sheffield-united', 'southampton-fc', 'tottenham-hotspur', 'west-bromwich-albion', 'west-ham-united', 'wolverhampton-wanderers']
chn_name = ['阿森纳', '阿斯顿维拉', '布莱顿', '伯恩利', '切尔西', '水晶宫', '埃弗顿', '富勒姆', '利兹联', '莱斯特城', '利物浦', '曼城', '曼联', '纽卡斯尔联', '谢菲尔德联', '南安普敦', '热刺', '西布朗', '西汉姆联', '狼队']
com_name = ['/eng-premier-league-', '/eng-fa-cup-', '/eng-league-cup-', '/europa-league-', '/champions-league-']

In [54]:
def get_tables(url):
    #res = requests.get(url)
    res = ''
    while res == '':
      try:
        res = requests.get(url)
        break
      except:
        print("Connection refused by the server..")
        print("Sleep for 5 seconds")
        time.sleep(5)
        print("Was a nice sleep, now let me continue...")
        continue
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.select('table.standard_tabelle')
    return table

In [55]:
def get_time(tables, ind, dic):
    time = []
    
    for player in tables[0].findAll('tr'):
        count = 0
        # process each record in row at a time
        for t in player.findAll('td'):
            if count == 0:
              name = t.text
            if count == 1:
              time = t.text.rstrip("'")
              if name in dic:
                if time.isdigit():
                  dic[name].append(int(time))
                else:
                  dic[name].append(0)
              else:
                if time.isdigit():
                  dic[name] = [int(time)]
                else:
                  dic[name] = [0]
            count += 1

    #df = pd.DataFrame.from_dict(dic, orient='index', columns = ['Time', 'Squad_eng', 'Squad'])
    return dic

In [58]:
for i in range(20):
  dic = dict()
  sumdic = dict()
  for j in range(5):
    url = 'https://www.worldfootball.net/team_performance/'+eng_name[i]+com_name[j]+'2020-2021'
    table = get_tables(url)
    dic = get_time(table, i, dic)
  val = list(dic.values())
  key = list(dic.keys())
  for k in range(len(val)):
    for j in range(len(val[k])):
      sum = 0
      sum = sum + val[k][j]
    #dic[key[k]].append(tot)
    sumdic[key[k]] = [eng_name[i], chn_name[i], sum]
  df_new = pd.DataFrame.from_dict(sumdic, orient='index', columns = ['Squad_eng', 'Squad', 'Time'])
  if i == 0:
    df = df_new
  else:
    df = pd.concat([df, df_new])
df.to_excel('complete_playingtime.xlsx')
print('done')

Connection refused by the server..
Sleep for 5 seconds
Was a nice sleep, now let me continue...
done
